In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
data = pd.read_csv('../outputs/17/out17.csv', index_col='Unnamed: 0')
triggers = pd.read_csv('../outputs/17/triggers17.csv', index_col='label')
reg_eff = pd.read_csv('../outputs/model/outmodel.csv', index_col='Unnamed: 0')

In [3]:
data.head()

,Nsig,dNsig,lum,Emean,dEmax,dEmin
641.0,308.24,18.63,3283.2,641.02,-0.0,0.0
650.0,92.55,10.42,932.0,649.77,0.0,0.0
675.0,80.24,9.44,1295.2,674.85,-0.0,0.0
700.0,34.82,6.38,917.9,700.56,0.0,-0.0
725.0,63.00,7.94,1155.9,724.94,0.0,-0.0


In [4]:
triggers.head()

,TF,CF,TC,eTF,eCF,TrigEff,vTF,vCF,TrigErr
label,,,,,,,,,
641.0,83.0,4.0,256.0,0.98,0.75,1.00,0.01,0.02,0.00
650.0,13.0,0.0,86.0,0.99,0.86,1.00,0.01,0.03,0.00
675.0,25.0,3.0,64.0,0.94,0.71,0.98,0.03,0.05,0.01
700.0,10.0,1.0,30.0,0.94,0.74,0.98,0.04,0.07,0.01
725.0,13.0,2.0,48.0,0.94,0.78,0.99,0.03,0.05,0.01


In [5]:
reg_eff.head()

,Nsig,dNsig,lum,Emean,dEmax,dEmin
550.0,1716.34,43.47,7163,550.0,0.0,0.0
562.5,1497.20,40.99,6306,562.5,0.0,0.0
575.0,1312.69,37.49,5516,575.0,0.0,0.0
587.5,1228.37,36.67,5116,587.5,0.0,0.0
600.0,1061.35,33.64,4606,600.0,0.0,0.0


Get N/L

In [6]:
data['CSvis'] = data['Nsig']/data['lum']
data['dCSvis'] = data['dNsig']/data['lum']

Trigger join

In [7]:
data = data.join(triggers[['TrigEff', 'TrigErr']], how='left')

In [8]:
data = data.drop(['Nsig', 'dNsig', 'lum'], axis=1)

Get Reg_eff

In [9]:
reg_eff['RegEff'] = reg_eff['Nsig']/reg_eff['lum']
reg_eff['RegErr'] = reg_eff['dNsig']/reg_eff['lum']
reg_eff.drop(['dEmax', 'dEmin', 'lum', 'Nsig', 'dNsig'], axis=1, inplace=True)

Reg eff join

In [10]:
data['RegEff'] = data.apply(lambda x: np.interp(x['Emean'], reg_eff.Emean, reg_eff.RegEff), axis=1)
data['RegErr'] = data.apply(lambda x: np.interp(x['Emean'], reg_eff.Emean, reg_eff.RegErr), axis=1)

Recalc cross section

In [11]:
data['CS'] = data['CSvis']/data['TrigEff']/data['RegEff'];
data['dCS'] = data['CS']*np.sqrt( np.square( data['dCSvis']/data['CSvis'] ) + 
                                np.square( data['TrigErr']/data['TrigEff'] ) + 
                                np.square( data['RegErr']/data['RegEff'] ))

In [12]:
data.drop(['CSvis', 'dCSvis', 'TrigEff', 'TrigErr', 'RegEff', 'RegErr'], axis=1, inplace=True)

In [13]:
data.to_csv('../outputs/17/result.csv')